In [ ]:
!pip install pycryptodome
import cv2
import string
import os
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import hashlib
import matplotlib.pyplot as plt

In [ ]:
image_path=input("Enter the Image path: ")
x=cv2.imread(image_path)

# Convert BGR to RGB for displaying with matplotlib
xrgb=cv2.cvtColor(x,cv2.COLOR_BGR2RGB)
plt.imshow(xrgb)
plt.axis('off')
plt.show()

print("\nImage Shape(Hight,Width,Channels):",x.shape)

In [90]:
# Encryption and Decryption Functions

def derive_key(userkey):
  return hashlib.sha256(userkey.encode()).digest()[:16]

def encrypt_message(msg,userkey):
  key = derive_key(userkey)
  cipher = AES.new(key, AES.MODE_CBC)
  ct=cipher.encrypt(pad(msg.encode(), AES.block_size))
  return cipher.iv+ct

def decrypt_message(cipher_bytes,userkey):
  key = derive_key(userkey)
  iv = cipher_bytes[:16]
  ct=cipher_bytes[16:]
  cipher = AES.new(key, AES.MODE_CBC, iv)
  return unpad(cipher.decrypt(ct), AES.block_size).decode()

In [91]:
# Byte Mapping Dictionaries (Not used in current LSB)
d={}
c={}
for i in range(256):
  d[chr(i)]=i
  c[i]=chr(i)

In [ ]:
# Get User Input
text=input("Enter Secret Message: ")
key=input("Enter Key: ")

In [ ]:
# Encrypt Message with AES
encrypted_bytes=encrypt_message(text,key)
print("Encrypted Message:",encrypted_bytes)
# Note: 'l' is not the correct length for extraction later, use encrypted_len
encrypted_len=len(encrypted_bytes)
print(f"Encrypted Message length(in Byte): {encrypted_len}")

In [94]:
# Embed Encrypted Message into Image using LSB

x_enc=x.copy()

n,m,z=0,0,0
# Use the length of the encrypted message for embedding
encrypted_len=len(encrypted_bytes)
for i in range(encrypted_len):
  char_val=encrypted_bytes[i]
  for bit_pos in range(8):
    bit=(char_val>>(7-bit_pos))&1
    org_val=x_enc[n,m,z]
    x_enc[n,m,z]=(org_val & 254)|bit
    z=(z+1)%3
    if z==0:
      m+=1
      if m==x_enc.shape[1]:
        m=0
        n=n+1

In [ ]:
# Saving the Stego Image
cv2.imwrite("encrypted_image.jpg",x_enc)

print("Data successfully Hidden and saved as'encrypted_image.jpg'")

In [ ]:
# Extraction or the Decryption
kl, n, m, z = 0, 0, 0, 0
key1 = input("Enter the Key to extract the Data: ")

if key == key1:
  extracted_bytes = bytearray()
  # Use encrypted_len for the loop limit
  for i in range(encrypted_len):
    val = 0
    for bit_pos in range(8):
      # Extract the least significant bit
      bit = x_enc[n, m, z] & 1
      val = (val << 1) | bit
      z = (z + 1) % 3
      if z == 0:
        m += 1
        if m == x_enc.shape[1]:
          m = 0
          n = n + 1
    # Append the extracted byte
    extracted_bytes.append(val)

  try:
    # Decrypt the extracted bytes
    decrypted_message = decrypt_message(bytes(extracted_bytes), key)
    print("Decrypted Message:", decrypted_message)
  except Exception as e:
    print(f"Error during decryption: {e}")

else:
  print("Key Mismatched")